## This notebook analyzes the known ISOs and produces their origins on the celestial sphere

It uses the following methodology:
- Given orbital elements from JPL small-body database
- Calculate the (RA, Dec) of the incoming velocity vector
- Plot the results on a Mollweide projection

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time

Initializing orbital elements from JPL small-body database for known ISOs

In [6]:
oumuamua_elements = {
    'a': 1.27234500742808e+01 * u.au,  # Semi-major axis
    'e': 1.201133796102373e+01,      # Eccentricity
    'i': 122.7417062847286 * u.deg,  # Inclination
    'Omega': 24.59690955523242 * u.deg,  # Longitude of ascending node
    'omega': 241.8105360304898 * u.deg,  # Argument of perihelion
    'M': 51.1576197938249 * u.deg,  # Mean anomaly
    'epoch': Time('2458006.007321375231', format='jd', scale='utc')  # Epoch of the elements
}

borisov_elements = {
    'a': -0.8514922551937886 * u.au,  # Semi-major axis
    'e': 3.356475782676596,      # Eccentricity
    'i': 44.05264247909138 * u.deg,  # Inclination
    'Omega': 308.1477292269942 * u.deg,  # Longitude of ascending node
    'omega': 209.1236864378081 * u.deg,  # Argument of perihelion
    'M': 34.4294703072178 * u.deg,  # Mean anomaly
    'epoch': Time('2458826.052845906059', format='jd', scale='utc')  # Epoch of the elements
}

atlas_elements = {
    'a': -0.263915917517816 * u.au,  # Semi-major axis
    'e': 6.139587836355706,      # Eccentricity
    'i': 175.1131015287974 * u.deg,  # Inclination
    'Omega': 322.1568699043938 * u.deg,  # Longitude of ascending node
    'omega': 128.0099421020839 * u.deg,  # Argument of perihelion
    'M': -723.1822479798254 * u.deg,  # Mean anomaly
    'epoch': Time('2460977.981439259343', format='jd', scale='utc')  # Epoch of the elements
}

We want to compute the incoming velocity vector $v_{\infty}$ for each ISO. With that, we can compute the RA and Dec of the incoming direction.

In [26]:
def orbital_elements_to_ra_dec(e: float, a: u.Quantity, i_deg: u.Quantity, Omega_deg: u.Quantity, omega_deg: u.Quantity, is_barycentric=False) -> tuple[float, float, np.ndarray, np.ndarray]:
    '''
    Convert orbital elements to RA and Dec of the incoming velocity vector if an ISO.

    Parameters
    ----------
    e : float
        Eccentricity of the orbit.
    a : float
        Semi-major axis of the orbit (in AU).
    i_deg : float
        Inclination of the orbit (in degrees).
    Omega_deg : float
        Longitude of the ascending node (in degrees).
    omega_deg : float
        Argument of perihelion (in degrees).
    is_barycentric : bool
        If True, convert to barycentric coordinates.

    Returns
    -------
    ra : float
        Right Ascension of the incoming velocity vector (in degrees).
    dec : float
        Declination of the incoming velocity vector (in degrees).
    '''
    # Defining constants
    mu = 1.32712440018e11  # Gravitational parameter for the Sun

    # Convert angles from degrees to radians
    i = i_deg.to(u.rad).value
    Omega = Omega_deg.to(u.rad).value
    omega = omega_deg.to(u.rad).value

    # Convert semi-major axis to meters
    a = a.to(u.km).value

    # Hyperbola: e > 1, a < 0
    # Asymptote true anomaly magnitude:
    # cos(nu_inf) = -1/e
    nu_inf = np.arccos(-1.0 / e)      # in radians
    nu_in = -nu_inf                   # incoming branch (before perihelion)

    # Specific angular momentum (magnitude) h = sqrt(mu * a * (e^2 - 1))
    # For hyperbola a < 0 so mu * a * (e^2 - 1) is positive
    h = np.sqrt(mu * a * (e**2 - 1.0))

    # Distance at true anomaly nu:
    r = (a * (1.0 - e**2)) / (1.0 + e * np.cos(nu_in))   # km (will be large)
    
    # Radial and transverse velocities in perifocal frame:
    v_r = (mu / h) * e * np.sin(nu_in)
    v_t = (mu / h) * (1.0 + e * np.cos(nu_in))

    # Position in perifocal frame
    r_pf = np.array([r * np.cos(nu_in), r * np.sin(nu_in), 0.0])

    # Velocity in perifocal frame
    v_pf = np.array([v_r * np.cos(nu_in) - v_t * np.sin(nu_in),
                     v_r * np.sin(nu_in) + v_t * np.cos(nu_in),
                     0.0])
    
    # Rotation matrix from perifocal -> ECI (ICRS/ecliptic depending on elements)
    # R = R_z(Omega) @ R_x(i) @ R_z(omega) (3-1-3 rotation)
    Rz_Omega = np.array([
        [np.cos(Omega), -np.sin(Omega), 0.0],
        [np.sin(Omega),  np.cos(Omega), 0.0],
        [0.0,         0.0,        1.0]
    ])

    Rx_i = np.array([
        [1.0, 0.0,        0.0],
        [0.0, np.cos(i), -np.sin(i)],
        [0.0, np.sin(i),  np.cos(i)]
    ])

    Rz_omega = np.array([
        [np.cos(omega), -np.sin(omega), 0.0],
        [np.sin(omega),  np.cos(omega), 0.0],
        [0.0,         0.0,        1.0]
    ])

    R = Rz_Omega @ Rx_i @ Rz_omega

    r_eci = R @ r_pf   # km
    v_eci = R @ v_pf   # km/s

    # Incoming velocity vector is v_eci (points toward Sun for incoming object).
    # Radiant is opposite direction (the direction it came from)
    v_unit = v_eci / np.linalg.norm(v_eci)
    radiant_unit = -v_unit

    # Convert to RA, Dec (ICRS) using cartesian -> skycoord
    # Coordinates expect units; astropy's SkyCoord.from_cartesian wants cartesian representation.
    print(radiant_unit)

    sc = SkyCoord(x=radiant_unit[0], y=radiant_unit[1], z=radiant_unit[2],
                  representation_type='cartesian', frame='icrs')
    
    ra = sc.spherical.lon.deg
    dec = sc.spherical.lat.deg
    return ra, dec, v_eci, r_eci

In [27]:
ra, dec, v_eci, r_eci = orbital_elements_to_ra_dec(oumuamua_elements['e'], oumuamua_elements['a'], oumuamua_elements['i'], oumuamua_elements['Omega'], oumuamua_elements['omega'])
print("Radiant RA, Dec (deg):", ra, dec)

[-0.64037535 -0.61680802  0.45767596]
Radiant RA, Dec (deg): 223.92605118611536 27.237242729679654
